 
# DATA PRE-PROCESSING  

In [1]:
import cv2,os

data_path=r'dataset_with_mask'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'001': 0, '002': 1, '003': 2, '004': 3, '005': 4, '006': 5, '007': 6}
['001', '002', '003', '004', '005', '006', '007']
[0, 1, 2, 3, 4, 5, 6]


In [2]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [4]:
np.save('data',data)
np.save('target',new_target)

# Convolutional Neural Network Architecture


In [5]:
import numpy as np

data=np.load(r'data.npy')
target=np.load(r'target.npy')


In [9]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(7,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [11]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
59/59 [==============================] - ETA: 0s - loss: 1.7659 - accuracy: 0.2887

INFO:tensorflow:Assets written to: model-001.model\assets


INFO:tensorflow:Assets written to: model-001.model\assets


59/59 [==============================] - 80s 1s/step - loss: 1.7659 - accuracy: 0.2887 - val_loss: 1.4623 - val_accuracy: 0.4731
Epoch 2/20
59/59 [==============================] - ETA: 0s - loss: 1.2013 - accuracy: 0.5495

INFO:tensorflow:Assets written to: model-002.model\assets


...

INFO:tensorflow:Assets written to: model-003.model\assets


59/59 [==============================] - 79s 1s/step - loss: 0.8726 - accuracy: 0.7005 - val_loss: 1.1204 - val_accuracy: 0.6581
Epoch 4/20
59/59 [==============================] - ETA: 0s - loss: 0.7417 - accuracy: 0.7371

INFO:tensorflow:Assets written to: model-004.model\assets


INFO:tensorflow:Assets written to: model-004.model\assets


59/59 [==============================] - 78s 1s/step - loss: 0.7417 - accuracy: 0.7371 - val_loss: 0.9335 - val_accuracy: 0.7247
Epoch 5/20
59/59 [==============================] - 75s 1s/step - loss: 0.5982 - accuracy: 0.8016 - val_loss: 1.0650 - val_accuracy: 0.6688
Epoch 6/20
59/59 [==============================] - 75s 1s/step - loss: 0.5009 - accuracy: 0.8226 - val_loss: 0.9789 - val_accuracy: 0.6688
Epoch 7/20
59/59 [==============================] - 74s 1s/step - loss: 0.4154 - accuracy: 0.8629 - val_loss: 0.9841 - val_accuracy: 0.7355
Epoch 8/20
59/59 [==============================] - 75s 1s/step - loss: 0.3152 - accuracy: 0.8914 - val_loss: 0.9920 - val_accuracy: 0.7161
Epoch 9/20
59/59 [==============================] - 74s 1s/step - loss: 0.3284 - accuracy: 0.8844 - val_loss: 1.0492 - val_accuracy: 0.7398
Epoch 10/20
59/59 [==============================] - 75s 1s/step - loss: 0.2671 - accuracy: 0.9005 - val_loss: 1.0160 - val_accuracy: 0.7484
Epoch 11/20
59/59 [===========

# Detection 

In [12]:
from keras.models import load_model
import cv2
import numpy as np

In [13]:
from keras.models import load_model
model = load_model(r'E:\SIT\Face_Mask_Detector-master\model-015.model')

face_clsfr=cv2.CascadeClassifier(r'E:\SIT\Face_Mask_Detector-master\haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(2)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [14]:
video_capture = cv2.VideoCapture(0)

while(True):
    ret, img = video_capture.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
video_capture.release()

1/1 [==============================] - 0s 20ms/step


In [ ]:
source.release()